In [1]:
!pip install pyngrok flask fastapi uvicorn nest_asyncio language_tool_python mlflow langcode

INFO: pip is looking at multiple versions of notebook to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.3 

In [2]:
! pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple lingowiz==2.2.2

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 40.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
INFO: pip is looking at multiple versions of py3langid to determine which version is compatible with othe

In [3]:
!pip show lingowiz

Name: lingowiz
Version: 2.2.2
Summary: A package for MarianMT MultiLangual model inference
Home-page: https://github.com/yourusername/my_inference_package
Author: Patrick Saade
Author-email: 
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: accelerate, bert-score, bleu, camel-tools, datasets, evaluate, farasapy, fasttext, huggingface-hub, langcodes, language-tool-python, lxml, mlflow, py3langid, python-dotenv, sacrebleu, scikit-learn, sentencepiece, swifter, torch, tqdm, transformers
Required-by: 


In [4]:
import nest_asyncio
from google.colab import userdata
from pyngrok import conf,ngrok
from pydantic import BaseModel
from fastapi import FastAPI,Query
from typing import Optional
import uvicorn
import mlflow
import langcodes
from transformers import MarianTokenizer, MarianMTModel


In [13]:
# Mlflow
nest_asyncio.apply()
mlflow.environment_variables.MLFLOW_TRACKING_INSECURE_TLS = "true"
mlflow.environment_variables.MLFLOW_TRACKING_SERVER_CERT_PATH  = ""
get_ipython().system_raw("mlflow server --host 0.0.0.0 --port 5000 &")
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = userdata.get('ngrok_token')


# ngrok
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{5000}/\"")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
 * ngrok tunnel "NgrokTunnel: "https://6e2c-35-203-165-151.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000/"


In [6]:
from datetime import datetime

# Upload dummy model on MlFlow
experiment_name ="translator_French_Arabic_spec"
now = datetime.now()
formatted_date_time = now.strftime("%Y-%m-%d_%H:%M")
run_name = experiment_name+"_"+str(formatted_date_time)
hf_account="patrick844/"
token = userdata.get('HF_TOKEN')
model_name = "translator_French_Arabic_spec"
run_name = experiment_name+"_"+str(formatted_date_time)
path = hf_account + model_name
model = MarianMTModel.from_pretrained(path, token=token)
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-ar",token=token)
model.save_pretrained("model")
tokenizer.save_pretrained("model")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_name=run_name):
  mlflow.transformers.log_model("model","trained_model",task="translation")

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/305M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/827k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/925k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.19M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62418]], 'forced_eos_token_id': 0}
2024/11/28 08:02:52 INFO mlflow.tracking.fluent: Experiment with name 'translator_French_Arabic_spec' does not exist. Creating a new experiment.
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
2024/11/28 08:02:57 WARNING mlflow.tran

🏃 View run translator_French_Arabic_spec_2024-11-28_08:02 at: http://127.0.0.1:5000/#/experiments/804097160707414924/runs/ce4b57f4095743808d3daa1fcedd8d29
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/804097160707414924


In [7]:
from datetime import datetime

# Upload dummy model on MlFlow
experiment_name ="translator_English_Arabic_spec"
now = datetime.now()
formatted_date_time = now.strftime("%Y-%m-%d_%H:%M")
run_name = experiment_name+"_"+str(formatted_date_time)
hf_account="patrick844/"
token = userdata.get('HF_TOKEN')
model_name = "translator_English_Arabic_spec"
run_name = experiment_name+"_"+str(formatted_date_time)
path = hf_account + model_name
model = MarianMTModel.from_pretrained(path, token=token)
tokenizer = MarianTokenizer.from_pretrained("patrick844/translator_English_Arabic_spec",token=token)
model.save_pretrained("model_en")
tokenizer.save_pretrained("model_en")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_name=run_name):
  mlflow.transformers.log_model("model_en","trained_model",task="translation")

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}
2024/11/28 08:03:17 INFO mlflow.tracking.fluent: Experiment with name 'translator_English_Arabic_spec' does not exist. Creating a new experiment.
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
2024/11/28 08:03:18 WARNING mlflow.tra

🏃 View run translator_English_Arabic_spec_2024-11-28_08:03 at: http://127.0.0.1:5000/#/experiments/556171306495575882/runs/edc3eab3f2594e60940c56e267f32316
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/556171306495575882


In [8]:
from datetime import datetime

# Upload dummy model on MlFlow
experiment_name ="translator_Romanian_English_spec"
now = datetime.now()
formatted_date_time = now.strftime("%Y-%m-%d_%H:%M")
run_name = experiment_name+"_"+str(formatted_date_time)
hf_account="patrick844/"
token = userdata.get('HF_TOKEN')
model_name = "translator_English_Arabic_spec"
run_name = experiment_name+"_"+str(formatted_date_time)
path = hf_account + model_name
model = MarianMTModel.from_pretrained(path, token=token)
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-roa-en",token=token)
model.save_pretrained("model_en")
tokenizer.save_pretrained("model_en")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_name=run_name):
  mlflow.transformers.log_model("model_en","trained_model",task="translation")

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/800k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/779k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}
2024/11/28 08:03:24 INFO mlflow.tracking.fluent: Experiment with name 'translator_Romanian_English_spec' does not exist. Creating a new experiment.
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
2024/11/28 08:03:25 WARNING mlflow.transformers: The model card could not be retrieved from the hub due to [Errno 21] Is a directory: 'model_en'
2024/11/28 08:03:25 WARNING mlflow.transformers: Unable to find license information for this

🏃 View run translator_Romanian_English_spec_2024-11-28_08:03 at: http://127.0.0.1:5000/#/experiments/712011914623597482/runs/03b1c0002bf94042929273a1cde3decd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/712011914623597482


In [14]:
from lingowiz.inference import inference
mlflow.set_tracking_uri("http://127.0.0.1:5000")


text_en_long = "34 year old  male  patient .medically free before .there is no  previous h/o any chronic ilness .presented in ER with c/o right sided weakness , slurred speech and ataxia for today morning .patient is unable to walk, significant sign=o/e.he is conscious , oriented.was found high bp  in the ER .THERE IS RIGHT SIDED WEAKNESS IN BOTH UPPER AND LOWER LIMB MORE IN UPPER  limb.dysarthia and ataxia .abd =distended .no ll edema ..patient is planned for ct brain to exclude any  acute ischemic insult .cbc creatinin .ct brain show=Small hypodense areas of CSF density seen, one in right periventricular white matter, two in basal ganglia regions and one in body of corpus callosum, appearing old lacunar infarcts, for clinical correlation and MRI brain if acute ischemic insult suspected.patient reviwed by neurology consultent asked for icu admission for the patient as case of acute stroke, for asprin 81, plavex 75mg, tovast40mg, iv fluid, pantozol 40mg, ecg, dvt pumb..., other conditions"
text_ro_long = "Drumul învățării de noi abilități, fie într-un context profesional, fie personal, poate fi atât plin de satisfacții, cât și provocator. Oamenii pornesc adesea pe acest drum cu entuziasm, motivați de perspectiva creșterii și îmbunătățirii. Totuși, pe măsură ce progresează, pot întâlni obstacole, de la constrângeri de timp la curbele de învățare abrupte pe care multe abilități le implică. În ciuda acestor provocări, recompensele depășesc adesea luptele. Dezvoltarea expertizei într-un domeniu nou poate deschide uși către oportunități care erau anterior inaccesibile, permițând indivizilor să își lărgească orizonturile. De exemplu, cineva care învață o limbă nouă ar putea călători și comunica mai eficient, dobândind perspective asupra diferitelor culturi. În mod similar, un profesionist care stăpânește o tehnologie nouă poate deveni mai competitiv în industrie, îmbunătățindu-și perspectivele de angajare. În cele din urmă, urmărirea cunoașterii și dezvoltarea abilităților este o investiție care aduce beneficii pe viață, adăugând profunzime caracterului și îmbogățind experiențele. Cu perseverență, eșecurile devin trepte de succes, conducând la realizări semnificative și împlinitoare"

text_en = "Hello my name is coffe do you want to have lunch ?"
text_ro = "Astăzi este o zi frumoasă"

a,b,c = inference(text_ro_long)
d,e,f = inference(text_ro)
g,h,i = inference(text_en)
j,k,l = inference(text_en_long)

Detecting Language ...
FastText model loaded.

Indirect Translation Romanian to English to Arabic

Processing Text...
Finding Model ...

Indirect ...

Translating ...
Confidence Score: 0.67
Confidence Score: 0.58
['The path of learning new skills either in a professional or personal context can be as full of satisfaction as it is challenging people often start on this road with enthusiasm motivated by the prospect of growth and improvement however as they progress can encounter obstacles from time constraints to abrupt learning curves that many skills involve despite these challenges the rewards often exceed the struggles of the development of expertise in a new field can open doors to opportunities that were previously inaccessible allowing individuals to broaden their horizons for example someone who learns a new language could travel and communicate more', 'ومسار تعلم مهارات جديدة إما في سياق مهني أو شخصي يمكن أن يكون مليئاً بالرضا بقدر ما هو صعب لأن الناس كثيراً ما يبدأون على هذا ا

Confidence Score: 0.46
HEREEEEEEEE    2
Checking Error ...

Number of Errors: 0 
Sentence Length: 5 
Rating: 6.457273960113525
Logging ...
Addindg data to inference_data.csv
🏃 View run translator_Romanian_English_spec_2024-11-28_08:03 at: http://127.0.0.1:5000/#/experiments/712011914623597482/runs/03b1c0002bf94042929273a1cde3decd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/712011914623597482
Checking Error ...



Number of Errors: 0 
Sentence Length: 4 
Rating: 4.631997909810808
Logging ...
Addindg data to inference_data.csv
🏃 View run translator_English_Arabic_spec_2024-11-28_08:18 at: http://127.0.0.1:5000/#/experiments/556171306495575882/runs/f68b6d389aac447caaf197be7c8d87a5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/556171306495575882
Detecting Language ...
FastText model loaded.
Direct Translation English to Arabic

Processing Text ...
Finding Model ...
Translating ...
Confidence Score: 0.50
HEREEEEEEEE    1
Checking Error ...



Number of Errors: 0 
Sentence Length: 5 
Rating: 4.993797846966319
Logging ...
Addindg data to inference_data.csv
🏃 View run translator_English_Arabic_spec_2024-11-28_08:18 at: http://127.0.0.1:5000/#/experiments/556171306495575882/runs/f68b6d389aac447caaf197be7c8d87a5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/556171306495575882
Logging Complete
Detecting Language ...
FastText model loaded.
Direct Translation English to Arabic

Processing Text...
Finding Model ...
Translating ...
Confidence Score: 0.47
['قبل أن لا يكون هناك تاريخ سابق لأي حالة مرضية مزمنة تعرض في غرفة الطوارئ مع شكاوي من الضعف الجانبي الأيمن، الخطاب الملتوي والاتكسيا لمريض الصباح اليوم غير قادر على المشي، علامة كبيرة على الفحص الذي هو واع، تم العثور على ارتفاع ضغط الدم في غرفة الطواريء']

Processing Text...
Finding Model ...
Translating ...
Confidence Score: 0.45
['قبل أن لا يكون هناك تاريخ سابق لأي حالة مرضية مزمنة تعرض في غرفة الطوارئ مع شكاوي من الضعف الجانبي الأيمن، الخطاب الملتوي والاتكسيا لمريض ا

Addindg data to inference_data.csv
🏃 View run translator_English_Arabic_spec_2024-11-28_08:18 at: http://127.0.0.1:5000/#/experiments/556171306495575882/runs/f68b6d389aac447caaf197be7c8d87a5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/556171306495575882
Logging Complete


In [11]:
from lingowiz.inference import inference

# Load model directly
from transformers import pipeline
nest_asyncio.apply()

from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Load AraBERT
# def correction(text):
#     tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")
#     model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv02")

#     # Example Arabic sentence with a mistake
#     sentence = "المريضة تركت ألماً نقصاً عضلياً مع الكثير من تَقَلُّب الغاز"

#     # Tokenize the sentence
#     inputs = tokenizer(sentence, return_tensors="pt")

#     # Get predictions
#     with torch.no_grad():
#         outputs = model(**inputs)
#         predictions = torch.argmax(outputs.logits, dim=-1)

#     # Decode the predicted tokens
#     predicted_sentence = tokenizer.decode(predictions[0], skip_special_tokens=True)
#     print(f"Original: {sentence}")
#     print(f"Corrected: {predicted_sentence}")
#     return predicted_sentence

class MlflowRequest(BaseModel):
  run_name:str
  model_name:str


print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = userdata.get('ngrok_token')

# Initialize FastAPI app
app = FastAPI()

# Open a ngrok tunnel to the HTTP server
domain = "ideal-amoeba-specially.ngrok-free.app"
public_url = ngrok.connect(4000, domain="ideal-amoeba-specially.ngrok-free.app").public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{4000}/\"")


@app.post("/mlflow")
async def log_model(request_body:MlflowRequest):
  # Download Model from HuggingFace
  hf_account="patrick844/"
  token = userdata.get('HF_TOKEN')
  model_name = request_body.model_name
  run_name = request_body.run_name
  path = hf_account + model_name
  model = MarianMTModel.from_pretrained(path, token=token)
  tokenizer = MarianTokenizer.from_pretrained(path,token=token)
  model.save_pretrained("model")
  tokenizer.save_pretrained("model")

  mlflow.set_tracking_uri("http://127.0.0.1:5000")
  mlflow.set_experiment(model_name)

  #Upload model on mlflow
  run_id = mlflow.search_runs(filter_string=f"run_name='{run_name}'")['run_id'][0]
  with mlflow.start_run(run_id = run_id):
    mlflow.transformers.log_model("model","trained_model",task="translation")


@app.get("/inference")
async def text_to_arabic(input: Optional[str] = Query(None)):
    import json
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    translation_list = []
    source_list=[]

    text, language, number_errors = inference(input,translation_list,source_list)
    # text = correction(text)
    source_language = language
    language_code = langcodes.find(source_language).language
    print(f"Language Code Inference mlflow: {language_code}")
    experiment_name=f"translation_{language_code}"
    result = {"translation": text, "language": language,"errors_count":number_errors}
    return result

# , authorizationid: Optional[str] = Query(None)
@app.get("/inferencestr")
async def text_to_arabic(input: Optional[str] = Query(None), id: Optional[str] = Query(None)):
    import json

    text=""
    language="English"
    number_errors=1

    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    # print("authorization_id ", authorizationid)
    print("id ", id)
    print("input ", input)
    return_obj = {
        "id":id,
        "input":input

    }
    # Handeling Special Cases
    if input == "new-visit":
      text = "زيارة جديدة"
      return_obj["translation"] = text
      return_obj["language"] = language
      return_obj["error_count"] = number_errors


    elif input=="referral":
      text = "إحالة"
      return_obj["translation"] = text
      return_obj["language"] = language
      return_obj["error_count"] = number_errors

    else:
      print("Original: ",input)
      translation_list = []
      source_list=[]
      text, language, number_errors = inference(input,translation_list, source_list)

      for translation in text:
        if translation:
          print(f"Translation: {translation}")
      if len(text) == 1:
        temp = text[0]
        text[0] = None
        text.append(temp)

        language.append(None)



      return_obj["translation"] = text
      return_obj["language"] = language
      return_obj["error_count"] = number_errors[0]


    return return_obj

# Run the server using Uvicorn
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=4000)


Loading the FastText language detection model...
Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken


INFO:     Started server process [5608]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4000 (Press CTRL+C to quit)


 * ngrok tunnel "https://ideal-amoeba-specially.ngrok-free.app" -> "http://127.0.0.1:4000/"


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}
2024/11/28 08:18:51 WARNING mlflow.transformers: The model card could not be retrieved from the hub due to [Errno 21] Is a directory: 'model'
2024/11/28 08:18:51 WARNING mlflow.transformers: Unable to find license information for this model. Please verify permissible usage for the model you are storing prior to use.
2024/11/28 08:18:51 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI.

🏃 View run translator_English_Arabic_spec_2024-11-28_08:18 at: http://127.0.0.1:5000/#/experiments/556171306495575882/runs/f68b6d389aac447caaf197be7c8d87a5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/556171306495575882
INFO:     34.29.215.201:0 - "POST /mlflow HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5608]
